# Polluting Weather
by Will Rosenfeld

# Abstract
Objective: To understand the role weather plays in driving air quality levels in Los Angeles.

Data:
    1. EPA Air Quality Index Daily Values Report (2008-2017): 
        - Location: https://www.epa.gov/outdoor-air-quality-data/air-quality-index-report
        - Documentation: https://www.epa.gov/outdoor-air-quality-data/about-air-data-reports#aqidaily
    2. NOAA US Daily Weather Data (2010-2017): 
        - Location: https://www.epa.gov/outdoor-air-quality-data/air-quality-index-daily-values-report
        - Documentation: https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
Method:
    - Target: Air Quality Index (https://en.wikipedia.org/wiki/Air_quality_index#United_States) will serve as a strong predictor variable
    - Predictors: Temp, Precip, Wind, Clouds, etc.

# Imports

In [1]:
# General
import pandas as pd
import numpy as np
import math
from datetime import datetime
from sklearn import metrics
# Selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
# Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# Vizualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [2]:
# Import EPA Daily Air Quality Index data for Los Angeles
pol_path = '/Users/wtrosenfeld/Google Drive/WTR Personal Files/Learning/Post Grad/General Assembly/ds-dc-22/5_final-projects/Polluting Weather/Data/EPA/'
la08 = pd.DataFrame.from_csv(pol_path + 'aqidaily2008.csv')
la09 = pd.DataFrame.from_csv(pol_path + 'aqidaily2009.csv')
la10 = pd.DataFrame.from_csv(pol_path + 'aqidaily2010.csv')
la11 = pd.DataFrame.from_csv(pol_path + 'aqidaily2011.csv')
la12 = pd.DataFrame.from_csv(pol_path + 'aqidaily2012.csv')
la13 = pd.DataFrame.from_csv(pol_path + 'aqidaily2013.csv')
la14 = pd.DataFrame.from_csv(pol_path + 'aqidaily2014.csv')
la15 = pd.DataFrame.from_csv(pol_path + 'aqidaily2015.csv')
la16 = pd.DataFrame.from_csv(pol_path + 'aqidaily2016.csv')
la17 = pd.DataFrame.from_csv(pol_path + 'aqidaily2017.csv')
#perhaps use instead: pd.read_csv(url, index_col='datetime', parse_dates=True)

frames = [la08,la09,la10,la11,la12,la13,la14,la15,la16,la17]
pol = pd.concat(frames)

In [3]:
# Import NOAA Daily Weather data for Los Angeles
wea_path = '/Users/wtrosenfeld/Google Drive/WTR Personal Files/Learning/Post Grad/General Assembly/ds-dc-22/5_final-projects/Polluting Weather/Data/NOAA/'
wea = pd.DataFrame.from_csv(wea_path + '1106788.csv')

In [4]:
# Import EPA Historical Next Day AQI Forecasts for Los Angeles
pfr_path = '/Users/wtrosenfeld/Google Drive/WTR Personal Files/Learning/Post Grad/General Assembly/ds-dc-22/5_final-projects/Polluting Weather/Data/EPA/'
pfr = pd.DataFrame.from_csv(pfr_path + 'aqinextdayforecast2010_2017.csv')

# Data Preparation

In [5]:
# Reformat AQI data
pol.reset_index(inplace=True)
pol.columns = ['DATE','MAIN_POLLUTANT','AQI','CO','SO2','NO2','OZONE','PM10','PM25']
print(pol.shape)

(3469, 9)


In [6]:
# Reformat and Filter Weather data
wea.reset_index(inplace=True)
wea = wea[wea['STATION'] == 'USW00093134']
wea['DATE'] = pd.to_datetime(wea['DATE'])

In [7]:
# Reformat and filter forecast data
pfr.reset_index(inplace=True)
pfr = pfr[pfr['ForecastPeriod'] == 'Next Day']
pfr['DATE'] = pd.to_datetime(pfr['ForecastDate'])
pfr = pfr.pivot(index='DATE', columns='Parameter', values='AQI')
pfr.reset_index(inplace=True)
pfr['AQI_fore'] = pfr[['CO','NO2','OZONE','PM10','PM2.5']].max(axis=1)
pfr.drop(['CO','NO2','OZONE','PM10','PM2.5'], axis=1, inplace=True)
print(pfr.shape)

(2872, 2)


In [8]:
# Join pollution and weather data
pw = pd.merge(wea,pol,how='inner',left_on=wea['DATE'],right_on=pol['DATE'])
pw["DATE"] = pw['DATE_x']
pw.drop(['DATE_x','DATE_y'], axis=1, inplace=True)
print(pw.shape)

(3469, 66)


In [9]:
# Left join pollution forecast data
pw = pd.merge(pw,pfr,how='left', on='DATE')
print(pw.shape)

(3469, 67)
